# Evaluación en conjunto de entrenamiento

En este notebook se va a realizar el código para la evaluacion de los tres modelos entrenados. Para esto se sacarán las métricas $mAP50$ y $mAP50-95$ directamente sobre el conjunto de entrenamiento, así como generar intervalos de confianza para estas utilizando el método de Bootstrap.

In [1]:
from ultralytics import YOLO
import pandas as pd


model_v11n = YOLO('ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0/weights/best.pt')
model_v11s = YOLO('ENTRENAMIENTO_YOLOV11S/YOLOV11S_AdamW_lr0.0005_wd0.0005_mos0/weights/best.pt')
model_v10s = YOLO('ENTRENAMIENTO_YOLOV10S/YOLOV10S_AdamW_lr0.0005_wd0.0005_mos0/weights/best.pt')

In [8]:
metrics_yolov11n = model_v11n.val(
    data=r'/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml',
    split='test',
    workers=0,      
    verbose=False,
    save=False,
    plots=False,
    project="EVALUACION",
    name="YOLOV11N",
    device="mps",
    batch=64
)

Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 203.0±58.9 MB/s, size: 42.9 KB)


val: Scanning /Users/pepefv97/Downloads/DATOS_PLACAS/test/labels.cache... 1019 i
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all       1019       1048      0.985      0.972      0.991      0.708
Speed: 0.3ms preprocess, 2.3ms inference, 0.0ms loss, 4.1ms postprocess per image


In [9]:
metrics_yolov11s = model_v11s.val(
    data=r'/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml',
    split='test',
    workers=0,      
    verbose=False,
    save=False,
    plots=False,
    project="EVALUACION",
    name="YOLOV11S",
    device="mps",
    batch=64
)

Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 207.8±138.2 MB/s, size: 45.8 KB)


val: Scanning /Users/pepefv97/Downloads/DATOS_PLACAS/test/labels.cache... 1019 i
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all       1019       1048       0.99      0.962      0.988      0.721
Speed: 0.2ms preprocess, 3.4ms inference, 0.0ms loss, 4.5ms postprocess per image


In [10]:
metrics_yolov10s = model_v10s.val(
    data=r'/Users/pepefv97/Downloads/DATOS_PLACAS/data.yaml',
    split='test',
    workers=0,      
    verbose=False,
    save=False,
    plots=False,
    project="EVALUACION",
    name="YOLOV10S",
    device="mps",
    batch=64
)

Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 265.8±224.9 MB/s, size: 43.8 KB)


val: Scanning /Users/pepefv97/Downloads/DATOS_PLACAS/test/labels.cache... 1019 i
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all       1019       1048      0.977      0.961      0.987      0.714
Speed: 0.2ms preprocess, 5.4ms inference, 0.0ms loss, 0.5ms postprocess per image


In [11]:
metricas = {
    'Precision': metrics_yolov11n.box.mp,
    'Recall': metrics_yolov11n.box.mr,
    'mAP@0.5': metrics_yolov11n.box.map50,
    'mAP@0.5:0.95': metrics_yolov11n.box.map,
    'Tiempo Preprocesamiento (ms/img)': metrics_yolov11n.speed['preprocess'],
    'Tiempo Inferencia (ms/img)': metrics_yolov11n.speed['inference'],
    'Tiempo Postprocesamiento (ms/img)': metrics_yolov11n.speed['postprocess'],
    'Número de clases detectadas': metrics_yolov11n.box.nc
}

df_metrics = pd.DataFrame([metricas])
df_metrics.to_csv("EVALUACION/YOLOV11N/metrics_test.csv", index=False)

In [12]:
metricas_10s = {
    'Precision': metrics_yolov10s.box.mp,
    'Recall': metrics_yolov10s.box.mr,
    'mAP@0.5': metrics_yolov10s.box.map50,
    'mAP@0.5:0.95': metrics_yolov10s.box.map,
    'Tiempo Preprocesamiento (ms/img)': metrics_yolov10s.speed['preprocess'],
    'Tiempo Inferencia (ms/img)': metrics_yolov10s.speed['inference'],
    'Tiempo Postprocesamiento (ms/img)': metrics_yolov10s.speed['postprocess'],
    'Número de clases detectadas': metrics_yolov10s.box.nc
}

df_metrics_10s = pd.DataFrame([metricas_10s])
df_metrics_10s.to_csv("EVALUACION/YOLOV10S/metrics_test.csv", index=False)

In [13]:
metricas_11s = {
    'Precision': metrics_yolov11s.box.mp,
    'Recall': metrics_yolov11s.box.mr,
    'mAP@0.5': metrics_yolov11s.box.map50,
    'mAP@0.5:0.95': metrics_yolov11s.box.map,
    'Tiempo Preprocesamiento (ms/img)': metrics_yolov11s.speed['preprocess'],
    'Tiempo Inferencia (ms/img)': metrics_yolov11s.speed['inference'],
    'Tiempo Postprocesamiento (ms/img)': metrics_yolov11s.speed['postprocess'],
    'Número de clases detectadas': metrics_yolov11s.box.nc
}

df_metrics_11s = pd.DataFrame([metricas_11s])
df_metrics_11s.to_csv("EVALUACION/YOLOV11S/metrics_test.csv", index=False)

# Generación de intervalos de confianza para las métricas.

In [1]:

import random, pathlib, os, shutil, yaml, uuid

B  = 1000                              
TEST_IMG = pathlib.Path("/Users/pepefv97/Downloads/DATOS_PLACAS/test/images")   
TEST_LAB = pathlib.Path("/Users/pepefv97/Downloads/DATOS_PLACAS/test/labels")   
BOOT_ROOT = pathlib.Path("BOOTSTRAP/SETS")
CLASS_NAME = ["PLACA_AUTOMOVIL"] 
ROOT_PATH = pathlib.Path("/Users/pepefv97/Downloads/PROYECTO_5")
imgs = sorted(TEST_IMG.glob("*.jpg")) 
                                



In [6]:
random.seed(123)                               

for b in range(B):
    set_dir  = ROOT_PATH / BOOT_ROOT / f"set_{b:04d}"
    img_dir  = ROOT_PATH / set_dir / "test/images"
    lab_dir  = ROOT_PATH / set_dir / "test/labels"
    img_dir.mkdir(parents=True, exist_ok=True)
    lab_dir.mkdir(parents=True, exist_ok=True)

    sample = random.choices(imgs, k=len(imgs))  
    for idx, src in enumerate(sample):
        tag      = f"{idx:04d}_{uuid.uuid4().hex[:6]}"
        dst_img  = img_dir / f"{tag}_{src.name}"
        dst_lab  = lab_dir / f"{tag}_{src.stem}.txt"

        os.symlink(src.resolve(), dst_img)      # enlace a la imagen
        os.symlink((TEST_LAB / f"{src.stem}.txt").resolve(), dst_lab)

    yaml_path = set_dir / "data.yaml"
    yaml_path.write_text(yaml.dump({
        "names": CLASS_NAME,
        "nc"   : len(CLASS_NAME),
        "train" : str(img_dir),
        "val" : str(img_dir),
        "test" : str(img_dir)
    }))


In [ ]:
import pathlib, time, pandas as pd
from ultralytics import YOLO

BOOT_SETS = ROOT_PATH / BOOT_ROOT 
OUT_ROOT = ROOT_PATH / pathlib.Path("BOOTSTRAP")     
DEVICE = "mps"                            
BATCH = 64                               


MODELS = {
    "YOLOV11N": "ENTRENAMIENTO_YOLOV11/YOLOV11_AdamW_lr0.001_wd1e-4_mos1.0/weights/best.pt",
    "YOLOV11S": "ENTRENAMIENTO_YOLOV11S/YOLOV11S_AdamW_lr0.0005_wd0.0005_mos0/weights/best.pt",
    "YOLOV11M": "ENTRENAMIENTO_YOLOV10S/YOLOV10S_AdamW_lr0.0005_wd0.0005_mos0/weights/best.pt"
}

yaml_files = sorted(BOOT_SETS.glob("set_*/data.yaml"))
assert yaml_files, "No se encontraron réplicas en BOOTSTRAP/SETS/"

rows = []
for model_name, model_path in MODELS.items():
    print(f"Evaluando {model_name} …")
    model = YOLO(model_path)               

    for yml in yaml_files:
        start = time.time()
        r = model.val(
            data=str(yml),
            split="test",
            batch=BATCH,
            device=DEVICE,
            workers=0,      
            verbose=False,
            save=False,
            plots=False
        )

        rows.append({
            "Model"   : model_name,
            "Sample"  : yml.parent.name,     
            "Precision": r.box.mp,
            "Recall"   : r.box.mr,
            "mAP50"    : r.box.map50,
            "mAP5095"  : r.box.map,
            "Pre_ms"   : r.speed["preprocess"],
            "Inf_ms"   : r.speed["inference"],
            "Post_ms"  : r.speed["postprocess"],
            "Elapsed_s": round(time.time() - start, 2)
        })

    out_dir = OUT_ROOT / model_name
    out_dir.mkdir(exist_ok=True)
    (pd.DataFrame([r for r in rows if r["Model"] == model_name])
        .drop(columns="Model")
        .to_csv(out_dir / "bootstrap_metrics.csv", index=False))

print("CSV listos en BOOTSTRAP/<MODEL>/bootstrap_metrics.csv")


Evaluando YOLOV11N …
Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
YOLO11n summary (fused): 100 layers, 2,582,347 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 156.9±47.4 MB/s, size: 41.9 KB)


val: Scanning /Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/SETS/set_0000/test/

val: New cache created: /Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/SETS/set_0000/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  m

                   all       2038       2110      0.986      0.971      0.988      0.709
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 4.2ms postprocess per image
Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1606.1±1146.0 MB/s, size: 48.5 KB)


...


val: Scanning /Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/SETS/set_0375/test/
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all       1019       1053      0.977      0.965       0.99      0.707
Speed: 0.1ms preprocess, 5.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1183.3±925.5 MB/s, size: 37.5 KB)



val: Scanning /Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/SETS/set_0376/test/
                 Class     Images  Instances      Box(P          R      mAP50  m

                   all       1019       1040      0.983      0.973      0.993      0.727
Speed: 0.1ms preprocess, 5.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Ultralytics 8.3.169 🚀 Python-3.12.2 torch-2.7.1 MPS (Apple M4 Pro)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1879.6±1122.9 MB/s, size: 46.8 KB)



val: Scanning /Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/SETS/set_0377/test/
                 Class     Images  Instances      Box(P          R      mAP50  m

In [2]:
import pandas as pd
from pathlib import Path

csv_files = {
    "YOLOv11-n": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV11N/bootstrap_metrics.csv"),
    "YOLOv11-s": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV11S/bootstrap_metrics.csv"),
    "YOLOv10-s": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV10S/bootstrap_metrics.csv"),
}

# Métricas numéricas sobre las que quieres intervalos
metric_cols = [
    "Precision", "Recall",
    "mAP50", "mAP5095",
    "Pre_ms", "Inf_ms", "Post_ms"
]

rows = []
for model, path in csv_files.items():
    df = pd.read_csv(path, usecols=metric_cols)
    ci_low  = df.quantile(0.025)
    ci_high = df.quantile(0.975)
    median  = df.median()

    for col in metric_cols:
        rows.append({
            "Model": model,
            "Metric": col,
            "Median": median[col],
            "CI_low": ci_low[col],
            "CI_high": ci_high[col]
        })

summary = pd.DataFrame(rows)

# -------------------------------------------------------------
# 2) Guarda el DataFrame (o imprímelo en pantalla)
# -------------------------------------------------------------
#summary.to_csv("BOOTSTRAP/bootstrap_ci.csv", index=False)
print(summary)


        Model     Metric    Median    CI_low   CI_high
0   YOLOv11-n  Precision  0.986206  0.976414  0.994114
1   YOLOv11-n     Recall  0.972302  0.956934  0.982776
2   YOLOv11-n      mAP50  0.991045  0.985557  0.994154
3   YOLOv11-n    mAP5095  0.708129  0.695594  0.720516
4   YOLOv11-n     Pre_ms  0.202544  0.185116  0.224151
5   YOLOv11-n     Inf_ms  1.930448  1.900192  2.022237
6   YOLOv11-n    Post_ms  4.152065  3.915525  4.305477
7   YOLOv11-s  Precision  0.989753  0.975083  0.996032
8   YOLOv11-s     Recall  0.965484  0.949763  0.979846
9   YOLOv11-s      mAP50  0.987967  0.980703  0.993697
10  YOLOv11-s    mAP5095  0.721978  0.708322  0.734341
11  YOLOv11-s     Pre_ms  0.134159  0.127809  0.145857
12  YOLOv11-s     Inf_ms  3.372617  3.337052  3.705755
13  YOLOv11-s    Post_ms  4.492810  4.245828  4.784184
14  YOLOv10-s  Precision  0.979069  0.967539  0.988732
15  YOLOv10-s     Recall  0.959924  0.944605  0.973052
16  YOLOv10-s      mAP50  0.987500  0.982454  0.992558
17  YOLOv1

In [6]:
import pandas as pd
from pathlib import Path

# Rutas a los CSV de réplicas bootstrap (uno por modelo)
csv_files = {
    "YOLOv11-n": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV11N/bootstrap_metrics.csv"),
    "YOLOv11-s": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV11S/bootstrap_metrics.csv"),
    "YOLOv10-s": Path("/Users/pepefv97/Downloads/PROYECTO_5/BOOTSTRAP/YOLOV10S/bootstrap_metrics.csv"),
}

rows = []
for model, path in csv_files.items():
    df = pd.read_csv(path)

    # ---------- tiempo total por imagen ----------
    df["Time_ms"] = df["Pre_ms"] + df["Inf_ms"] + df["Post_ms"]

    # Inference únicamente
    inf_q = df["Inf_ms"].quantile([0.025, 0.50, 0.975])
    # Total
    tot_q = df["Time_ms"].quantile([0.025, 0.50, 0.975])

    rows.append({
        "Model": model,
        "Inf_median":  inf_q.loc[0.50],
        "Inf_CI_low":  inf_q.loc[0.025],
        "Inf_CI_high": inf_q.loc[0.975],
        "Total_median":  tot_q.loc[0.50],
        "Total_CI_low":  tot_q.loc[0.025],
        "Total_CI_high": tot_q.loc[0.975],
    })

summary = pd.DataFrame(rows)
#summary.to_csv("BOOTSTRAP/bootstrap_time_ci_extended.csv", index=False)
print(summary)


       Model  Inf_median  Inf_CI_low  Inf_CI_high  Total_median  Total_CI_low  \
0  YOLOv11-n    1.930448    1.900192     2.022237      6.286433      6.046396   
1  YOLOv11-s    3.372617    3.337052     3.705755      8.010068      7.741971   
2  YOLOv10-s    5.264049    5.246526     5.321358      5.921241      5.879718   

   Total_CI_high  
0       6.481295  
1       8.586405  
2       5.982035  
